In [1]:
import csv
import pandas as pd

In [2]:
df =pd.read_csv('Refined data/KQ_Data.csv')
df

,Date3,Departures,Class,Review,Rating,Rating_Description
0,2022-09-01 00:00:00,Africa,Economy,Recently I travelled for business from Cape To...,4.0,Good
1,2022-11-01 00:00:00,International,Economy,I want to thanks to Jane for her excellent ser...,5.0,Excellent
2,2022-05-01 00:00:00,International,Economy,not a bad airline to fly with could do with a ...,4.0,Good
3,2022-11-01 00:00:00,Africa,Business Class,"Dear Kenya Airways ,\r\n\r\nI am on my way to ...",1.0,Poor
4,2022-11-01 00:00:00,Africa,Economy,"Absolutely appalling airline, never ever use i...",1.0,Poor
...,...,...,...,...,...,...
3060,26-01-2017,NaN,NaN,Better than I expected Much better service tha...,0.0,NaN
3061,30-08-2016,NaN,NaN,Request for immediate action. I have never had...,0.0,NaN
3062,02-07-2016,NaN,NaN,Never again Flew with Kenya airways last year ...,0.0,NaN
3063,21-11-2015,NaN,NaN,NEVER AGAIN - RUBISH COMPANY !!! We travelled ...,1.0,NaN


In [41]:
import streamlit as st
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
import pickle

# Initialize NLTK tools
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    return ' '.join(words)

def get_review_sentiment(review):
    sentiment_score = sia.polarity_scores(review)['compound']
    if sentiment_score > 0.1:
        return 'positive'
    elif sentiment_score < -0.1:
        return 'negative'
    else:
        return 'neutral'

def determine_final_sentiment(row):
    review_sentiment = get_review_sentiment(row['Review'])
    rating = row['Rating']
    
    if review_sentiment == 'positive' or rating >= 4:
        return 'positive'
    elif review_sentiment == 'negative' or rating <= 2:
        return 'negative'
    else:
        return 'neutral'

# Use Streamlit's new caching methods
@st.cache_data
def load_data():
    # Replace 'your_file.csv' with the path to your dataset
    df = pd.read_csv('Refined data/KQ_Data.csv')

    return df

df = load_data()

# Apply the function to determine sentiment
df['sentiment'] = df.apply(determine_final_sentiment, axis=1)

# Define features (X) and target (y)
X_reviews = df['Review']
X_ratings = df['Rating']
y = df['sentiment']

# Apply the preprocessing to the reviews
X_reviews = X_reviews.apply(preprocess_text)

# Initialize the vectorizer and fit/transform the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_reviews_transformed = vectorizer.fit_transform(X_reviews).toarray()

# Convert ratings into a 2D array with one feature (reshape for concatenation)
X_ratings_transformed = X_ratings.values.reshape(-1, 1)

# Concatenate the review features with the rating feature
X_combined = np.hstack((X_reviews_transformed, X_ratings_transformed))

# Handle missing values by imputing (if any)
imputer = SimpleImputer(strategy='mean')
X_combined = imputer.fit_transform(X_combined)

# Encode the target variable (sentiment)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Initialize SMOTE and apply it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize and fit the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = logistic_model.predict(X_test)
st.write(classification_report(y_test, y_pred))
st.write('Accuracy:', accuracy_score(y_test, y_pred))

# Save the model and vectorizer
with open('logistic_model.pkl', 'wb') as file:
    pickle.dump(logistic_model, file)

with open('vectorizer.pkl', 'wb') as file:
    pickle.dump(vectorizer, file)

st.write("Model and vectorizer saved successfully!")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
2024-08-16 15:53:28.839 No runtime found, using MemoryCacheStorageManager
2024-08-16 15:53:28.859 No runtime found, using MemoryCacheStorageManager


In [26]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

data = df

# Define features (X) and target (y)
X_reviews = data['Review']
X_ratings = data['Rating']
y = data['sentiment']

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('stopwords')

# Initialize the stemmer and stopwords
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# Function for text preprocessing
def preprocess_text(text):
    # Tokenize the text
    words = word_tokenize(text.lower())
    
    # Remove stopwords and perform stemming
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    
    return ' '.join(words)

# Apply the preprocessing to the reviews
X_reviews = X_reviews.apply(preprocess_text)

# Initialize the vectorizer and fit/transform the text data
vectorizerr= TfidfVectorizer(max_features=5000)  # Adjust max_features as needed
X_reviews_transformed = vectorizerr.fit_transform(X_reviews).toarray()

# Convert ratings into a 2D array with one feature (reshape for concatenation)
X_ratings_transformed = X_ratings.values.reshape(-1, 1)

# Concatenate the review features with the rating feature
X_combined = np.hstack((X_reviews_transformed, X_ratings_transformed))

# Handle missing values by imputing (if any)
imputer = SimpleImputer(strategy='mean')
X_combined = imputer.fit_transform(X_combined)

# Encode the target variable (sentiment)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Initialize SMOTE and apply it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize and fit the Random Forest model
best_rff= RandomForestClassifier(n_estimators=100, random_state=42)
best_rff.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = best_rff.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


              precision    recall  f1-score   support

           0       0.91      0.97      0.94       226
           1       0.95      0.95      0.95       218
           2       0.94      0.96      0.95       224
           3       0.99      0.90      0.94       236

    accuracy                           0.94       904
   macro avg       0.95      0.95      0.94       904
weighted avg       0.95      0.94      0.94       904

Accuracy: 0.9446902654867256


In [13]:
import pickle

# Save the model
with open('best_rff.pkl', 'wb') as file:
    pickle.dump(best_rff, file)

# Save the vectorizer
with open('vectorizerr.pkl', 'wb') as file:
    pickle.dump(vectorizerr, file)


In [42]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import FunctionTransformer

# Initialize NLTK tools
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    return ' '.join(words)

def get_review_sentiment(review):
    sentiment_score = sia.polarity_scores(review)['compound']
    if sentiment_score > 0.1:
        return 'positive'
    elif sentiment_score < -0.1:
        return 'negative'
    else:
        return 'neutral'

def determine_final_sentiment(row):
    review_sentiment = get_review_sentiment(row['Review'])
    rating = row['Rating']
    
    if review_sentiment == 'positive' or rating >= 4:
        return 'positive'
    elif review_sentiment == 'negative' or rating <= 2:
        return 'negative'
    else:
        return 'neutral'


# Apply the function to determine sentiment
df['sentiment'] = df.apply(determine_final_sentiment, axis=1)

# Define features (X) and target (y)
X_reviews = df['Review']
X_ratings = df['Rating']
y = df['sentiment']

# Apply the preprocessing to the reviews
X_reviews = X_reviews.apply(preprocess_text)

# Initialize the vectorizer and fit/transform the text data
vectorizer = TfidfVectorizer()
X_reviews_transformed = vectorizer.fit_transform(X_reviews).toarray()

# Convert ratings into a 2D array with one feature (reshape for concatenation)
X_ratings_transformed = X_ratings.values.reshape(-1, 1)

# Concatenate the review features with the rating feature
X_combined = np.hstack((X_reviews_transformed, X_ratings_transformed))

# Handle missing values by imputing (if any)
imputer = SimpleImputer(strategy='mean')
X_combined = imputer.fit_transform(X_combined)

# Encode the target variable (sentiment)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Initialize SMOTE and apply it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Initialize and fit the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000, random_state=42)
logistic_model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = logistic_model.predict(X_test)
print(classification_report(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


              precision    recall  f1-score   support

           0       0.85      0.92      0.88       346
           1       1.00      1.00      1.00       359
           2       0.92      0.85      0.89       398

    accuracy                           0.92      1103
   macro avg       0.92      0.92      0.92      1103
weighted avg       0.92      0.92      0.92      1103

Accuracy: 0.9211242067089755


In [43]:
import joblib

# Save the vectorizer
vectorizer_filename = 'tfidf_vectorizer.pkl'
joblib.dump(vectorizer, vectorizer_filename)
print(f"Vectorizer saved to {vectorizer_filename}")

# Save the logistic regression model
model_filename = 'logistic_regression_model.pkl'
joblib.dump(logistic_model, model_filename)
print(f"Model saved to {model_filename}")

# Save the label encoder
label_encoder_filename = 'label_encoder.pkl'
joblib.dump(le, label_encoder_filename)
print(f"Label encoder saved to {label_encoder_filename}")


Vectorizer saved to tfidf_vectorizer.pkl
Model saved to logistic_regression_model.pkl
Label encoder saved to label_encoder.pkl


In [39]:
with open('logistic_model.pkl', 'wb') as file:
    logistic_model = pickle.dump(logistic_model,file)

with open('tfidf_vectorizer.pkl', 'wb') as file:
    vectorizer = pickle.dump(tfidf_vectorizer,file)


In [29]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Initialize NLTK tools
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    return ' '.join(words)

def get_review_sentiment(review):
    sentiment_score = sia.polarity_scores(review)['compound']
    if sentiment_score > 0.1:
        return 'positive'
    elif sentiment_score < -0.1:
        return 'negative'
    else:
        return 'neutral'

def determine_final_sentiment(row):
    review_sentiment = get_review_sentiment(row['Review'])
    rating = row['Rating']
    
    if review_sentiment == 'positive' or rating >= 4:
        return 'positive'
    elif review_sentiment == 'negative' or rating <= 2:
        return 'negative'
    else:
        return 'neutral'

# Apply the function to determine sentiment
df['sentiment'] = df.apply(determine_final_sentiment, axis=1)

# Check the transformation
print(df[['Rating', 'Review', 'sentiment']].head())

# Define features (X) and target (y)
X_reviews = df['Review']
X_ratings = df['Rating']
y = df['sentiment']

# Apply the preprocessing to the reviews
X_reviews = X_reviews.apply(preprocess_text)

# Initialize the vectorizer and fit/transform the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_reviews_transformed = vectorizer.fit_transform(X_reviews).toarray()

# Convert ratings into a 2D array with one feature (reshape for concatenation)
X_ratings_transformed = X_ratings.values.reshape(-1, 1)

# Concatenate the review features with the rating feature
X_combined = np.hstack((X_reviews_transformed, X_ratings_transformed))

# Handle missing values by imputing (if any)
imputer = SimpleImputer(strategy='mean')
X_combined = imputer.fit_transform(X_combined)

# Encode the target variable (sentiment)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Initialize SMOTE and apply it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Define a function to train and evaluate models
def train_and_evaluate_model(df, model, text_column='Review', rating_column='Rating', target_column='sentiment'):
    """
    Function to preprocess text and numeric data, train a specified model, and evaluate its performance.
    """
    # Apply preprocessing and transformation steps as described above
    # Preprocessing and feature extraction should be done here

    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions and evaluate the model
    y_pred = model.predict(X_test)
    print(f"Model: {model.__class__.__name__}")
    print(classification_report(y_test, y_pred))
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print("-" * 60)

# Instantiate models
log_reg = LogisticRegression(max_iter=1000)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

# Train and evaluate each model
train_and_evaluate_model(df, log_reg)
train_and_evaluate_model(df, rf)
train_and_evaluate_model(df, xgb_model)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


   Rating                                             Review sentiment
0     4.0  Recently I travelled for business from Cape To...  positive
1     5.0  I want to thanks to Jane for her excellent ser...  positive
2     4.0  not a bad airline to fly with could do with a ...  positive
3     1.0  Dear Kenya Airways ,\r\n\r\nI am on my way to ...  negative
4     1.0  Absolutely appalling airline, never ever use i...  negative
Model: LogisticRegression
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       346
           1       1.00      1.00      1.00       359
           2       0.93      0.85      0.89       398

    accuracy                           0.92      1103
   macro avg       0.92      0.92      0.92      1103
weighted avg       0.93      0.92      0.92      1103

Accuracy: 0.9220308250226654
------------------------------------------------------------
Model: RandomForestClassifier
              precision    recall  f1-score   s

In [30]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Initialize NLTK tools
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    return ' '.join(words)

def get_review_sentiment(review):
    sentiment_score = sia.polarity_scores(review)['compound']
    if sentiment_score > 0.1:
        return 'positive'
    elif sentiment_score < -0.1:
        return 'negative'
    else:
        return 'neutral'

def determine_final_sentiment(row):
    review_sentiment = get_review_sentiment(row['Review'])
    rating = row['Rating']
    
    if review_sentiment == 'positive' or rating >= 4:
        return 'positive'
    elif review_sentiment == 'negative' or rating <= 2:
        return 'negative'
    else:
        return 'neutral'

# Load your dataset
data = df

# Apply the function to determine sentiment
data['sentiment'] = data.apply(determine_final_sentiment, axis=1)

# Define features (X) and target (y)
X_reviews = data['Review']
X_ratings = data['Rating']
y = data['sentiment']

# Apply the preprocessing to the reviews
X_reviews = X_reviews.apply(preprocess_text)

# Initialize the vectorizer and fit/transform the text data
vectorizer = TfidfVectorizer(max_features=5000)
X_reviews_transformed = vectorizer.fit_transform(X_reviews).toarray()

# Convert ratings into a 2D array with one feature (reshape for concatenation)
X_ratings_transformed = X_ratings.values.reshape(-1, 1)

# Concatenate the review features with the rating feature
X_combined = np.hstack((X_reviews_transformed, X_ratings_transformed))

# Handle missing values by imputing (if any)
imputer = SimpleImputer(strategy='mean')
X_combined = imputer.fit_transform(X_combined)

# Encode the target variable (sentiment)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Initialize SMOTE and apply it to the data
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_combined, y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

def train_and_evaluate_model(model, X_train, X_test, y_train, y_test):
    """
    Train and evaluate a model. Print classification metrics for classification models
    and RMSE for regression models.
    """
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    if hasattr(model, 'predict_proba'):  # Check if the model has predict_proba method
        print(f"Model: {model.__class__.__name__}")
        print(classification_report(y_test, y_pred))
        print('Accuracy:', accuracy_score(y_test, y_pred))
        print("-" * 60)
    else:
        # If model does not have predict_proba, assume it's a regression model
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        print(f"Model: {model.__class__.__name__}")
        print('RMSE:', rmse)
        print("-" * 60)

# Define a list of models to evaluate
models = [
    LogisticRegression(max_iter=1000),
    RandomForestClassifier(n_estimators=100, random_state=42),
    GradientBoostingClassifier(),
    XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
    SVC(probability=True),
    KNeighborsClassifier(n_neighbors=5)
]

# Train and evaluate each model
for model in models:
    train_and_evaluate_model(model, X_train, X_test, y_train, y_test)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Model: LogisticRegression
              precision    recall  f1-score   support

           0       0.84      0.93      0.88       346
           1       1.00      1.00      1.00       359
           2       0.94      0.85      0.89       398

    accuracy                           0.92      1103
   macro avg       0.93      0.93      0.92      1103
weighted avg       0.93      0.92      0.92      1103

Accuracy: 0.9238440616500453
------------------------------------------------------------
Model: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.83      0.93      0.88       346
           1       1.00      1.00      1.00       359
           2       0.94      0.84      0.88       398

    accuracy                           0.92      1103
   macro avg       0.92      0.92      0.92      1103
weighted avg       0.92      0.92      0.92      1103

Accuracy: 0.9202175883952856
------------------------------------------------------------
Mo

In [32]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from collections import Counter

# Initialize NLTK tools
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

def preprocess_text(text):
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text.lower())
    words = [ps.stem(w) for w in words if w not in stop_words and w.isalpha()]
    return ' '.join(words)

def get_review_sentiment(review):
    sentiment_score = sia.polarity_scores(review)['compound']
    if sentiment_score > 0.1:
        return 'positive'
    elif sentiment_score < -0.1:
        return 'negative'
    else:
        return 'neutral'

# Load your dataset
data = df

# Apply the function to determine sentiment
data['sentiment'] = data['Review'].apply(get_review_sentiment)

# Preprocess text for further analysis
data['Review_processed'] = data['Review'].apply(preprocess_text)

# Function to extract and display reviews with specific sentiment and keywords
def analyze_sentiment_reviews(data, sentiment_label, keywords=None):
    filtered_data = data[data['sentiment'] == sentiment_label]
    
    if keywords:
        keyword_reviews = filtered_data[filtered_data['Review_processed'].str.contains('|'.join(keywords))]
    else:
        keyword_reviews = filtered_data

    return keyword_reviews

# Example: Analyze reviews with negative sentiment and specific keywords
negative_keywords = ['bad', 'poor', 'terrible', 'worst', 'disappointing']
positive_keywords = ['good', 'excellent', 'fantastic', 'great', 'wonderful']
neutral_keywords = ['okay', 'average', 'fair', 'normal', 'typical']

# Extract and print reviews with negative sentiment and keywords
negative_reviews = analyze_sentiment_reviews(data, 'negative', negative_keywords)
print("Negative Reviews:")
print(negative_reviews[['Review']].head())

# Extract and print reviews with positive sentiment and keywords
positive_reviews = analyze_sentiment_reviews(data, 'positive', positive_keywords)
print("Positive Reviews:")
print(positive_reviews[['Review']].head())

# Extract and print reviews with neutral sentiment and keywords
neutral_reviews = analyze_sentiment_reviews(data, 'neutral', neutral_keywords)
print("Neutral Reviews:")
print(neutral_reviews[['Review']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Negative Reviews:
                                               Review
3   Dear Kenya Airways ,\r\n\r\nI am on my way to ...
7   After reading the reviews on here, I was prepa...
8   Kenya Airways is very disorganzied, first of a...
13  My Thai husband and I were planning on traveli...
14  My wife and myself flew from Nairobi to Zanzib...
Positive Reviews:
                                               Review
0   Recently I travelled for business from Cape To...
2   not a bad airline to fly with could do with a ...
5   We flew from Heathrow to Nairobi on our way to...
11  Great service and friendly staff most of all B...
15  Pleasantly surprised with this airline. Servic...
Neutral Reviews:
                                                 Review
1179  Food was not eatable. Media was not subtitled ...
1233  The Dreamliner flight from Nairobi to Amsterda...
1248  The flight was on time as per schedule but the...
2758  Heathrow to Harare return. Outbound flight was...
2841  Missed connec